# QDC CALIBRATION

In this notebook we obtain the ASICs qdc calibration using the tpulse.

We obtain a curve efine vs integration window, so that when taking real data we can correct the efine obtained for any integration window.

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pylab as plt
%matplotlib inline
import os

In [ ]:
import sys 
sys.path.append("/home/nsalor/Software/ANTEA")

In [ ]:
from antea.preproc.io              import get_files
from antea.preproc.io              import read_run_data
from antea.preproc.tdc_corrections import compute_integration_window_size
from antea.preproc.qdc_corrections import correct_efine_wrap_around
from antea.preproc.qdc_calibration import compute_qdc_calibration_using_mode
from antea.preproc.qdc_calibration import plot_all_channels
from antea.preproc.qdc_calibration import process_qdc_calibration_run

# Process data

In [ ]:
run_number = 12384

In [ ]:
pd.read_hdf('/analysis/12384/hdf5/data/run_12384_0000_trigger1_waveforms.h5', 'data', stop = 10000)

The function below shows two plots. The first one shows the entries for each channel. The second one the entries for each integration window.

In [ ]:
df_qdc = process_qdc_calibration_run(run_number, max_intg_w = 416, hist = True, folder = '/analysis/{run}/hdf5/data/')

In [ ]:
df_qdc

**Create the expected windows to filter wrong values**

In [ ]:
expected_windows = np.concatenate([[0], np.arange(6, 416, 5), [2000]])
expected_windows

In [ ]:
df_filtered = df_qdc[df_qdc.intg_w.isin(expected_windows)]

**Check max and min integration window size for each channel and tac to avoid wrong interpolations**

In [ ]:
df_filtered.groupby(['channel_id', 'tac_id']).min().intg_w.unique()

In [ ]:
df_filtered.groupby(['channel_id', 'tac_id']).max().intg_w.unique()

We show the efine vs integration window curves for all channels:

In [ ]:
plot_all_channels(df_filtered, [0, 220])
plt.xlabel('intg_w')
plt.ylabel('efine')
plt.grid()
plt.xticks(np.linspace(0, 300, 21), rotation = 30)

In [ ]:
df_ch7 = df_filtered[(df_filtered['channel_id'] == 7)]
plt.plot(df_ch7[df_ch7['tac_id'] == 0].intg_w, df_ch7[df_ch7['tac_id'] == 0].efine, label = 'tac0')
plt.plot(df_ch7[df_ch7['tac_id'] == 1].intg_w, df_ch7[df_ch7['tac_id'] == 1].efine, label = 'tac1')
plt.plot(df_ch7[df_ch7['tac_id'] == 2].intg_w, df_ch7[df_ch7['tac_id'] == 2].efine, label = 'tac2')
plt.plot(df_ch7[df_ch7['tac_id'] == 3].intg_w, df_ch7[df_ch7['tac_id'] == 3].efine, label = 'tac3')
#plt.xlim(0, 300)

plt.xlabel('intg_w')
plt.ylabel('efine')
plt.grid()

## **Save data in file**

In [ ]:
#df_filtered.to_hdf('qdc_calibration_cold_compressor_asic581_tofpet7_imirror25_version33_r12384.h5', 'data')